In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchtext.legacy import data
from torchtext.legacy import datasets

import random
import numpy as np

import time

In [2]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [3]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

tokenizer = nltk.RegexpTokenizer(r"\w+").tokenize #It will remove punctuation

TEXT = data.Field(tokenize=tokenizer, lower = True)
LABEL = data.LabelField()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [4]:

train_data, valid_data, test_data = datasets.SNLI.splits(TEXT, LABEL)

downloading snli_1.0.zip


snli_1.0.zip: 100%|██████████| 94.6M/94.6M [00:40<00:00, 2.34MB/s]


extracting


In [9]:
print(f"Number of training examples: {len(train_data)}")
print(f"Number of validation examples: {len(valid_data)}")
print(f"Number of testing examples: {len(test_data)}")

Number of training examples: 549367
Number of validation examples: 9842
Number of testing examples: 9824


In [10]:
print(vars(train_data.examples[0]))

{'premise': ['a', 'person', 'on', 'a', 'horse', 'jumps', 'over', 'a', 'broken', 'down', 'airplane'], 'hypothesis': ['a', 'person', 'is', 'training', 'his', 'horse', 'for', 'a', 'competition'], 'label': 'neutral'}


In [7]:
MIN_FREQ = 2

TEXT.build_vocab(train_data, 
                 min_freq = MIN_FREQ,
                 vectors = "glove.6B.300d",
                 unk_init = torch.Tensor.normal_)

LABEL.build_vocab(train_data)

.vector_cache/glove.6B.zip: 862MB [02:43, 5.29MB/s]                           
100%|█████████▉| 398980/400000 [00:37<00:00, 11022.83it/s]

In [8]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")

Unique tokens in TEXT vocabulary: 23352


In [11]:
print(TEXT.vocab.freqs.most_common(20))

[('a', 1439104), ('the', 534701), ('in', 407300), ('is', 373355), ('man', 266237), ('on', 235910), ('and', 206376), ('are', 199024), ('of', 192432), ('with', 169236), ('woman', 137630), ('two', 122259), ('people', 121159), ('to', 113972), ('at', 98662), ('wearing', 81024), ('an', 80212), ('his', 72467), ('shirt', 62744), ('young', 61515)]


In [12]:
print(TEXT.vocab.itos[:10])

['<unk>', '<pad>', 'a', 'the', 'in', 'is', 'man', 'on', 'and', 'are']


In [13]:
print(LABEL.vocab.itos)

['entailment', 'contradiction', 'neutral']


In [14]:
print(LABEL.vocab.freqs.most_common())

[('entailment', 183416), ('contradiction', 183187), ('neutral', 182764)]


In [15]:
BATCH_SIZE = 256

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    device = device)

In [88]:

class Model(nn.Module):
    def __init__(self, 
                 input_dim, 
                 embedding_dim,
                 hidden_dim,
                 n_fc_layers,
                 output_dim, 
                 dropout, 
                 pad_idx):
        
        super().__init__()
                                
        self.embedding = nn.Embedding(input_dim, embedding_dim, padding_idx = pad_idx)
        
        fc_dim = hidden_dim * 2
        
        fcs = [nn.Linear(fc_dim, fc_dim) for _ in range(n_fc_layers)]
        
        self.fcs = nn.ModuleList(fcs)
        
        self.fc_out = nn.Linear(fc_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, prem, hypo):

        prem_seq_len, batch_size = prem.shape
        hypo_seq_len, _ = hypo.shape

        #print('prem_seq_len: ', prem.size())
        
        #prem = [prem sent len, batch size]
        #hypo = [hypo sent len, batch size]
        
        embedded_prem = self.embedding(prem)
        embedded_hypo = self.embedding(hypo)

        #print('embedded_prem: ', embedded_prem.size())
        
        #embedded_prem = [prem sent len, batch size, embedding dim]
        #embedded_hypo = [hypo sent len, batch size, embedding dim]

        sum_prem = torch.sum(embedded_prem, dim=0)
        sum_hypo = torch.sum(embedded_hypo, dim=0)

        #print(sum_prem.size())

        #sum_prem = [batch_size, embedding_dim]
        #sum_hypo = [batch_size, embedding_dim]

        hidden = torch.cat((sum_prem, sum_hypo), dim=1)

        #hidden = [batch size, fc dim]
            
        for fc in self.fcs:
            hidden = fc(hidden)
            hidden = F.relu(hidden)
            hidden = self.dropout(hidden)
        
        prediction = self.fc_out(hidden)
        
        #prediction = [batch size, output dim]
        
        return prediction



In [89]:

INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 300
HIDDEN_DIM = 300
N_FC_LAYERS = 3
OUTPUT_DIM = len(LABEL.vocab)
DROPOUT = 0.25
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = Model(INPUT_DIM,
                  EMBEDDING_DIM,
                  HIDDEN_DIM,
                  N_FC_LAYERS,
                  OUTPUT_DIM,
                  DROPOUT,
                  PAD_IDX).to(device)


In [90]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 8,089,203 trainable parameters


In [91]:
pretrained_embeddings = TEXT.vocab.vectors

print(pretrained_embeddings.shape)

torch.Size([23352, 300])


In [92]:
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-0.1117, -0.4966,  0.1631,  ..., -1.4447,  0.8402, -0.8668],
        [ 0.1032, -1.6268,  0.5729,  ...,  0.3180, -0.1626, -0.0417],
        [-0.2971,  0.0940, -0.0967,  ...,  0.0597, -0.2285,  0.2960],
        ...,
        [-0.2149,  0.0846, -0.2949,  ...,  0.2379,  0.4804, -0.3348],
        [-0.0626, -0.3148, -0.9665,  ...,  1.3171, -0.8476, -0.4057],
        [ 0.3580, -0.0304,  0.3355,  ...,  0.0703, -0.5158,  0.1819]],
       device='cuda:0')

In [93]:

model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.embedding.weight.data)

tensor([[-0.1117, -0.4966,  0.1631,  ..., -1.4447,  0.8402, -0.8668],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.2971,  0.0940, -0.0967,  ...,  0.0597, -0.2285,  0.2960],
        ...,
        [-0.2149,  0.0846, -0.2949,  ...,  0.2379,  0.4804, -0.3348],
        [-0.0626, -0.3148, -0.9665,  ...,  1.3171, -0.8476, -0.4057],
        [ 0.3580, -0.0304,  0.3355,  ...,  0.0703, -0.5158,  0.1819]],
       device='cuda:0')


In [94]:
model.embedding.weight.requires_grad = False

In [95]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 1,083,603 trainable parameters


In [96]:
model = model.to(device)

In [97]:
optimizer = optim.Adam(model.parameters())

In [98]:

criterion = nn.CrossEntropyLoss().to(device)

In [99]:

def categorical_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    max_preds = preds.argmax(dim = 1, keepdim = True) # get the index of the max probability

    correct = (max_preds.squeeze(1)==(y)).float()

    return (correct.sum() / len(y)).to(device)

In [100]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        prem = batch.premise.to(device)
        hypo = batch.hypothesis.to(device)
        labels = batch.label.to(device)
        
        optimizer.zero_grad()
        
        #prem = [prem sent len, batch size]
        #hypo = [hypo sent len, batch size]
        
        predictions = model(prem, hypo)
        
        #predictions = [batch size, output dim]
        #labels = [batch size]
        
        loss = criterion(predictions, labels)
                
        acc = categorical_accuracy(predictions, labels)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [101]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            prem = batch.premise
            hypo = batch.hypothesis
            labels = batch.label
                        
            predictions = model(prem, hypo)
            
            loss = criterion(predictions, labels)
                
            acc = categorical_accuracy(predictions, labels)
            
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [102]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [105]:
N_EPOCHS = 10

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 14s
	Train Loss: 0.642 | Train Acc: 73.34%
	 Val. Loss: 0.628 |  Val. Acc: 74.15%
Epoch: 02 | Epoch Time: 0m 14s
	Train Loss: 0.637 | Train Acc: 73.57%
	 Val. Loss: 0.625 |  Val. Acc: 74.35%
Epoch: 03 | Epoch Time: 0m 14s
	Train Loss: 0.636 | Train Acc: 73.71%
	 Val. Loss: 0.625 |  Val. Acc: 74.49%
Epoch: 04 | Epoch Time: 0m 14s
	Train Loss: 0.634 | Train Acc: 73.76%
	 Val. Loss: 0.617 |  Val. Acc: 75.07%
Epoch: 05 | Epoch Time: 0m 14s
	Train Loss: 0.631 | Train Acc: 73.93%
	 Val. Loss: 0.614 |  Val. Acc: 75.17%
Epoch: 06 | Epoch Time: 0m 14s
	Train Loss: 0.628 | Train Acc: 73.99%
	 Val. Loss: 0.612 |  Val. Acc: 75.33%
Epoch: 07 | Epoch Time: 0m 14s
	Train Loss: 0.626 | Train Acc: 74.14%
	 Val. Loss: 0.621 |  Val. Acc: 75.07%
Epoch: 08 | Epoch Time: 0m 14s
	Train Loss: 0.625 | Train Acc: 74.16%
	 Val. Loss: 0.610 |  Val. Acc: 75.06%
Epoch: 09 | Epoch Time: 0m 14s
	Train Loss: 0.622 | Train Acc: 74.39%
	 Val. Loss: 0.613 |  Val. Acc: 75.22%
Epoch: 10 | Epoch T

In [106]:
model.load_state_dict(torch.load('model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} |  Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.615 |  Test Acc: 75.51%


In [107]:
def predict_inference(premise, hypothesis, text_field, label_field, model, device):
    
    model.eval()
    
    if isinstance(premise, str):
        premise = text_field.tokenize(premise)
    
    if isinstance(hypothesis, str):
        hypothesis = text_field.tokenize(hypothesis)
    
    if text_field.lower:
        premise = [t.lower() for t in premise]
        hypothesis = [t.lower() for t in hypothesis]
        
    premise = [text_field.vocab.stoi[t] for t in premise]
    hypothesis = [text_field.vocab.stoi[t] for t in hypothesis]
    
    premise = torch.LongTensor(premise).unsqueeze(1).to(device)
    hypothesis = torch.LongTensor(hypothesis).unsqueeze(1).to(device)
    
    prediction = model(premise, hypothesis)
    
    prediction = prediction.argmax(dim=-1).item()
    
    return label_field.vocab.itos[prediction]

In [108]:
premise = 'a man sitting on a green bench.'
hypothesis = 'a woman sitting on a green bench.'

predict_inference(premise, hypothesis, TEXT, LABEL, model, device)

'contradiction'

In [109]:
premise = 'a man sitting on a green bench.'
hypothesis = 'a man sitting on a blue bench.'

predict_inference(premise, hypothesis, TEXT, LABEL, model, device)

'contradiction'

In [110]:
premise = 'a man sitting on a green bench.'
hypothesis = 'a male sat on a lime bench.'

predict_inference(premise, hypothesis, TEXT, LABEL, model, device)

'neutral'

In [111]:
premise = 'a man sitting on a green bench.'
hypothesis = 'a person on a park bench'

predict_inference(premise, hypothesis, TEXT, LABEL, model, device)

'neutral'